In [11]:
import pandas as pd
import os

df_covs = pd.read_csv('processed_data/paul_covariates.csv')
df_covs.rename({'GEOID':'Fips'}, axis=1, inplace=True)
# df_covs

df_deaths = pd.read_csv('processed_data/naive_county_deaths.csv')
# df_deaths.drop(['Date','County','Pop'], axis=1, inplace=True)
# df_deaths

df = df_deaths.merge(df_covs, on='Fips')

In [22]:
import numpy as np
np.random.seed(666)

# # # # # # # # # #
test_ratio = 0.05
# # # # # # # # # #

###########################
# # FIPS SPLIT # # # # # #
###########################

fips = df.Fips.unique().tolist()

test_fips = np.random.choice(fips, int(test_ratio*len(fips)), replace=False)
train_fips = [x for x in fips if x not in test_fips]
print(len(test_fips), len(train_fips))

df_train = df[df.Fips.isin(train_fips)]
df_test = df[df.Fips.isin(test_fips)]
print(len(df_test), len(df_train))

###########################
# # DATE SPLIT # # # # # #
###########################

# dates = df.Date.unique().tolist()

# test_dates = np.random.choice(dates, int(test_ratio*len(dates)), replace=False)
# train_dates = [x for x in dates if x not in test_dates]
# # len(test_fips), len(train_fips)

# df_train = df[df.Date.isin(train_dates)]
# df_test = df[df.Date.isin(test_dates)]

5 95
195 3705


In [26]:
fips

[37001,
 37003,
 37005,
 37007,
 37009,
 37011,
 37013,
 37015,
 37017,
 37019,
 37021,
 37023,
 37025,
 37027,
 37029,
 37031,
 37033,
 37035,
 37037,
 37039,
 37041,
 37043,
 37045,
 37047,
 37049,
 37051,
 37053,
 37055,
 37057,
 37059,
 37061,
 37063,
 37065,
 37067,
 37069,
 37071,
 37073,
 37075,
 37077,
 37079,
 37081,
 37083,
 37085,
 37087,
 37089,
 37091,
 37093,
 37095,
 37097,
 37099,
 37101,
 37103,
 37105,
 37107,
 37109,
 37113,
 37115,
 37117,
 37111,
 37119,
 37121,
 37123,
 37125,
 37127,
 37129,
 37131,
 37133,
 37135,
 37137,
 37139,
 37141,
 37143,
 37145,
 37147,
 37149,
 37151,
 37153,
 37155,
 37157,
 37159,
 37161,
 37163,
 37165,
 37167,
 37169,
 37171,
 37173,
 37175,
 37177,
 37179,
 37181,
 37183,
 37185,
 37187,
 37189,
 37191,
 37193,
 37195,
 37197,
 37199]

In [24]:
target = 'True_county_deaths'

covs_to_drop = [target,
                'Fips',
                'County',
                'Date',
                'Pop'
#                 'Naive_county_deaths',
#                 'Pred_state_deaths'
               ]

X_train = df_train.drop(covs_to_drop,axis=1)
y_train = df_train[target].values

X_test = df_test.drop(covs_to_drop,axis=1)
y_test = df_test[target].values

y_naive = df_test['Naive_county_deaths'].values

print(len(X_train), len(X_test))


from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=666)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test) #[:,None]
y_prob = clf.predict_proba(X_test)[:,0]

# y_naive = X_test['Naive_county_deaths'].values

# print(y_test[:10])
# print(y_pred[:10])
# print(y_naive[:10])


from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

for which,pred in [ ['Naive', y_naive], ['RF',y_pred] ]:
    mae = round(mean_absolute_error(y_test, pred), 2)
    rmse = round(mean_squared_error(y_test, pred, squared=False), 2)
    r2 = round(r2_score(y_test, pred), 4)
    print(f"\n* {which} *")
    print(f"\tRMSE: {rmse} \tr2: {r2} \tMAE: {mae}")

3705 195

* Naive *
	RMSE: 379.14 	r2: -0.6793 	MAE: 214.95

* RF *
	RMSE: 170.51 	r2: 0.6604 	MAE: 119.08
